In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import pickle


df = pd.read_csv('train.csv')


drop_cols = [
    "Id", "MSSubClass", 'Street', 'Alley', 'LandContour', 'Utilities', 'LandSlope', 'Neighborhood',
    'Condition1', 'Condition2', 'MasVnrType', 'FireplaceQu', 'PavedDrive', '3SsnPorch', 'ScreenPorch',
    'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal', 'MoSold'
]
df.drop(columns=drop_cols, inplace=True, errors='ignore')


df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].mean())
df['BsmtQual'] = df['BsmtQual'].fillna(df['BsmtQual'].mode()[0])
df['MasVnrArea'] = df['MasVnrArea'].fillna(df['MasVnrArea'].mode()[0])

mode_cols = [
    'ExterQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical',
    'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond'
]
for col in mode_cols:
    df[col] = df[col].fillna(df[col].mode()[0])


for col in ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']:
    df[col] = df[col].astype('int64')


object_cols = df.select_dtypes(include=['object']).columns.tolist()
for col in object_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


df.to_csv('train.csv', index=False)


X = df.drop(columns=['SalePrice'])
y = df['SalePrice']


model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)


pickle.dump(model, open('pickle_Rf.pkl', 'wb'))

In [ ]:
sample_submission = pd.DataFrame({
    'Id': np.arange(1461, 1461 + len(y)),
    'SalePrice': y.values
})
sample_submission.to_csv('sample_submission.csv', index=False)